In [1]:
import cv2
import random
import glob
import numpy as np
import matplotlib.pyplot as plt
import random
import time 
import numpy as np
import skimage
import os 

from PIL import Image,ImageDraw,ImageFont,ImageEnhance,ImageFilter
%matplotlib inline

origin_images = glob.glob('./blank_region/*.jpg')
font = ImageFont.truetype('./font/Songti.ttc',20,index=6)
# font = ['./font/华文细黑.ttf','./font/Songti.ttc','./font/STHeitiLight.ttc']
save_dir = '/mnt/wuwenhui/git_ocr_project/chinese_ocr_densenet/train/images/num_dataset_v2/valid/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
font_dir = ['./font/STHeitiLight.ttc','./font/Songti.ttc','./font/Hiragino Sans GB.ttc']


In [ ]:
font_dir = ['./font/Songti.ttc','./font/华文细黑.ttf']
# font_dir = ['./font/华文细黑.ttf']

def SaltAndPepper(src,percetage):  
    SP_NoiseImg=src 
    SP_NoiseNum=int(percetage*src.shape[0]*src.shape[1]) 
    for i in range(SP_NoiseNum): 
        randX=np.random.randint(0,src.shape[0]-1) 
        randY=np.random.randint(0,src.shape[1]-1) 
        if np.random.randint(0,1)==0: 
            SP_NoiseImg[randX,randY]=0 
        else: 
            SP_NoiseImg[randX,randY]=255 
    return SP_NoiseImg

def image_enhance(img):
    
    flag1 = random.random()
    if flag1 > 0.3:
        img = ImageEnhance.Contrast(img).enhance(random.random()/2+0.8)
        img = ImageEnhance.Brightness(img).enhance(random.random()/1.5+0.75)
        flag2 = random.random()
        if flag2 > 0.5:
            for _ in range(random.randint(0,4)):
                img = ImageEnhance.Sharpness(img).enhance(0.01)
#                 print('sharp one')
        elif flag2 > 0.3:
            for _ in range(random.randint(0,3)):
                img = img.filter(ImageFilter.SMOOTH_MORE)
#                 print('smooth one')
    return img



def gen_text_randomSize(item):
    fontName= random.choice(font_dir)
    fontSize= random.randint(18,21)
    len_item = len(item.strip())
    origin_images = glob.glob('./blank_region/*.jpg')
    if fontName == './font/Songti.ttc':
        index = 6
    else:
        index = 0
#     print(fontName,fontSize)
    font = ImageFont.truetype(font=fontName,size=fontSize,index=index)
    img = cv2.imread(random.choice(origin_images))
    image = img.copy()
    h,w = img.shape[:2]
    blank = random.choice([2,2,2,2,2,25,50])
    text_h,text_w = random.randint(fontSize,fontSize+7),(fontSize-5)*len_item+1+blank
    if random.random() > 0.7:
        text_h,text_w = random.randint(fontSize+3,fontSize+7),(fontSize-5)*len_item+1+blank
    h_chop = h-text_h
    w_chop = w-text_w
#     print(h,w,text_h,text_w,blank)
    x = np.random.randint(w_chop)
    y = np.random.randint(h_chop)
    image = image[y:y+text_h,x:x+text_w]

    flag = random.random()
    if flag >0.7:
        image = skimage.util.random_noise(image,mode='gaussian',var=0.002)
        image = np.uint8(image*255)
    

#     print(image.shape)
    img_PIL = Image.fromarray(image[:,:,::-1])
    draw = ImageDraw.Draw(img_PIL)
    draw.text((blank,-0.5),item,font=font,fill=(0,0,0))
    
    img_PIL = image_enhance(img_PIL)
    
#     with open(save_dir+item.replace('/','&&')+ "-" + str(fontSize) +'.txt', "w", encoding='utf-8') as f:
#         f.write(item)
    img_PIL.save(save_dir+item.replace('/','&&') +'.png') 
    return img_PIL

In [ ]:
num_dict = '0123456789,./%-￥'
char_dict = 'ABCDEFGHIJKLMNPQRSTUVWXYZabcdefghijklmnpqrstuvwxyz'

def gen_number_with_dot():
    serial_len = random.randint(0,5)
    num = random.random() * 10**serial_len if random.random() > 0.5 else random.randint(1,10**serial_len)
    num =  num if random.random() > 0.5 else round(random.random(),random.randint(1,2)) 
    reserve_list = [ '{:,.1f}', '{:,.2f}', '{:,.3f}']
    serial = random.choice(reserve_list).format(num)
    return serial

def gen_number_only():
    serial_len = random.randint(0,5)
    num = random.randint(1,10**serial_len)
    return '{:,}'.format(num)

def gen_serial_num():
    serial_len = 10
    serial_begin = random.sample(char_dict[:]+num_dict[:10],1)
    serial = random.sample(char_dict[:]+num_dict[:],serial_len-1)
    return ''.join(serial_begin+serial)

def gen_money_with_dollar_sign():
    serial = gen_number_with_dot()
    serial_begin = '￥'
    return serial_begin+serial 

def gen_number_with_percent_sign():
    num = random.random() if random.random() > 0.5 else round(random.random(),2)
    reserve_list = ['{:.1%}','{:.2%}','{:.3%}']
    serial = random.choice(reserve_list).format(num)
    return serial


def gen_money_with_Chinese():
    serial = gen_number_with_dot()
    Chinese_list = ["合计:" + serial, serial+"元",'总额'+serial, '单价'+serial,"合计" + serial]
    return random.choice(Chinese_list)

def up_to_10(text):
    length = len(text)
    if length < 10:
        char_text = ''.join(random.sample(char_dict,10-length))
        text = char_text+text
    return text

In [ ]:
list_len = 200
serial_num = [gen_serial_num() for _ in range(list_len)]
number_only = [gen_number_only() for _ in range(list_len)]
number_with_dot = [gen_number_with_dot() for _ in range(list_len)]
money_with_dollar_sign = [gen_money_with_dollar_sign()  for _ in range(list_len)]
money_with_percent_sign = [gen_number_with_percent_sign()  for _ in range(list_len)]
random_char = [''.join(random.sample(num_dict,10)) for _ in range(list_len)]
money_with_Chinese = [gen_money_with_Chinese() for _ in range(list_len)]
total_num = number_with_dot + number_only + random_char + serial_num +\
            money_with_dollar_sign+money_with_percent_sign + money_with_Chinese + serial_num

total_num_10 = [up_to_10(text) for text in total_num]
random.shuffle(total_num)
len(total_num)

In [ ]:
# print(number_with_dot[:10])
# print(random_char[:10])
# print(number_only[:10])
# print(number_with_dot[:10])
# print(money_with_dollar_sign[:10])
# print(money_with_percent_sign[:10])
# print(money_with_Chinese[:10])
# print(serial_num[:10])
print(total_num_10[:20])

In [ ]:

# random_char = [''.join(random.sample(num_dict,10)) for _ in range(list_len)]
item = random.choice(total_num_10)
result = gen_text_randomSize(item)
plt.imshow(result)
item

In [ ]:
##### from multiprocessing import Pool,cpu_count

In [ ]:
from multiprocessing import Pool,cpu_count

items = set(total_num_10)
print(len(items))
pool = Pool(cpu_count()//4)
start = time.time()
%time result = pool.map(gen_text_randomSize,items)
end = time.time()
print(end-start,cpu_count()//4)



In [ ]:
number_with_dot = [gen_number_with_dot() for _ in range(30000)]
len(set(number_with_dot))

In [ ]:
def gen_text_randomSize_json_label(item):
    fontName= random.choice(font_dir)
    fontSize= random.randint(18,21)
    len_item = len(item.strip())
    origin_images = glob.glob('./blank_region/*.jpg')
    if fontName == './font/Songti.ttc':
        index = 6
    else:
        index = 0
#     print(fontName,fontSize)
    font = ImageFont.truetype(font=fontName,size=fontSize,index=index)
    img = cv2.imread(random.choice(origin_images))
    image = img.copy()
    h,w = img.shape[:2]
    blank = random.choice([2,2,2,2,2,25,50,80])
    text_h,text_w = random.randint(fontSize,fontSize+7),(fontSize-5)*len_item+1+blank
    if random.random() > 0.7:
        text_h,text_w = random.randint(fontSize+3,fontSize+7),(fontSize-5)*len_item+1+blank
    h_chop = h-text_h
    w_chop = w-text_w
#     print(h,w,text_h,text_w,blank)
    x = np.random.randint(w_chop)
    y = np.random.randint(h_chop)
    image = image[y:y+text_h,x:x+text_w]

    flag = random.random()
    if flag >0.7:
        image = skimage.util.random_noise(image,mode='gaussian',var=0.002)
        image = np.uint8(image*255)
    

#     print(image.shape)
    img_PIL = Image.fromarray(image[:,:,::-1])
    draw = ImageDraw.Draw(img_PIL)
    draw.text((blank,-0.5),item,font=font,fill=(0,0,0))
    
    img_PIL = image_enhance(img_PIL)
    
#     with open(save_dir+item.replace('/','&&')+ "-" + str(fontSize) +'.txt', "w", encoding='utf-8') as f:
#         f.write(item)
    img_PIL.save(save_dir+item.replace('/','&&')+ "-" + str(fontSize) +'.png') 
    return img_PIL

In [ ]:
import json

total_num_png = [item.replace('/','&&') +'.png' for item in total_num_10]
dict_num = dict(zip(total_num_png,total_num_10))


save_dir ='/mnt/wuwenhui/git_ocr_project/chinese_ocr_densenet/train/images/num_dataset_v2/'

#保存成.json
with open(save_dir+"valid_label.json",'w',encoding='utf-8') as json_file:   
        json.dump(dict_num,json_file)
        
# load json 文件
with open(save_dir+"valid_label.json",'r',encoding='utf-8') as json_file:
    txt=json.load(json_file)

In [ ]:
total_num_png = [item.replace('/','&&') +'.png' for item in total_num]
dict_num = dict(zip(total_num_png,total_num))

In [ ]:
len(txt),len(dict_num)

In [ ]:
dst_path = '/mnt/wuwenhui/git_ocr_project/chinese_ocr_densenet/train/images/medicine_dataset_v3/'

ori_path = '/mnt/wuwenhui/git_ocr_project/chinese_ocr_densenet/train/images/num_dataset_v2/'

# with open(dst_path+"train_label.json",'w',encoding='utf-8') as json_file:   
#         json.dump(dict_num,json_file)
        
# load json 文件
with open(dst_path+"valid_label.json",'r',encoding='utf-8') as json_file:
    dst_valid_txt=json.load(json_file)

    
with open(ori_path+"valid_label.json",'r',encoding='utf-8') as json_file:
    ori_valid_txt=json.load(json_file)

with open(dst_path+"train_label.json",'r',encoding='utf-8') as json_file:
    dst_train_txt=json.load(json_file)

    
with open(ori_path+"train_label.json",'r',encoding='utf-8') as json_file:
    ori_train_txt=json.load(json_file)
    

In [ ]:
len(dst_valid_txt),len(ori_valid_txt)

In [ ]:
total_train = {}
total_train.update(dst_valid_txt)
total_train.update(ori_valid_txt)
len(total_train),len(dst_valid_txt)


In [ ]:
with open(dst_path+"valid_label_total.json",'w',encoding='utf-8') as json_file:   
        json.dump(total_train,json_file)

In [ ]:
len(set(total_train.keys()) & set(imgs_png))

In [ ]:
import glob
imgs = glob.glob(dst_path+'valid/*png')
imgs2 = glob.glob(dst_path+'valid/.*.png')
file = glob.glob(ori_path+'train/*')
file2 = glob.glob(ori_path+'train/.*')

print(len(imgs+imgs2),len(file+file2))

imgs_png = set([os.path.split(img)[-1] for img in imgs+imgs2])


In [ ]:
remove_list = set(total_train.keys()) - set(imgs_png)

In [ ]:
num_dict = dict([(key,value) for key,value in total_train.items() if key not in remove_list])

In [ ]:
num_dict


with open(dst_path+"valid_label.json",'w',encoding='utf-8') as json_file:   
        json.dump(num_dict,json_file)

In [ ]:
len(num_dict)

In [19]:
save_dirsss = '/mnt/wuwenhui/git_ocr_project/chinese_ocr_densenet/train/images/dataset_len20_v1/valid/'
imgs = glob.glob(save_dirsss+'*jpg')
len(imgs)

17969

In [16]:
np.sort()

TypeError: sort() missing 1 required positional argument: 'a'

In [23]:
u'\x7f'.replace('\x7f','A')

'A'